# pγ→ηπ⁰p amplitude model

Formulate helicity amplitude model for $p \gamma \to \eta \pi^0 p$ symbolically using [AmpForm](https://ampform.rtfd.io).

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

In [ ]:
from collections import defaultdict

import ampform
import graphviz
import matplotlib.pyplot as plt
import qrules
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from ampform.io import aslatex, improve_latex_rendering
from IPython.display import Math
from qrules.particle import Particle, Spin, create_particle, load_pdg
from tensorwaves.data import (
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
)
from tensorwaves.function.sympy import create_parametrized_function

improve_latex_rendering()
particle_db = load_pdg()

## Generate transitions

In [ ]:
pgamma1 = Particle(
    name="pgamma1",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma2 = create_particle(
    template_particle=pgamma1,
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    pid=pgamma1.pid + 1,
)
particle_db.update([pgamma1, pgamma2])

We keep the relevant information about branching fractions in the previous [Branching fraction](../branching-fraction.md) chapter.
For simplicity, after the information about branching fraction, and under the current `qrules` results, we limit the intermediate resonances to be:

- $a(2)$ for $\eta \pi^0$, 
- $\Delta(1232)$ for $\pi p$",
- and "$N(1535)$" for both $\pi^0 p$ and $\eta p$.

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="pgamma1",
    final_state=["eta", "pi0", "p"],
    allowed_intermediate_particles=["a(2)(1320)", "N(1535)", "Delta(1232)"],
    allowed_interaction_types=["strong", "EM"],
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
src = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(src)

## Formulate model

In [ ]:
model_builder = ampform.get_builder(reaction)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = 0, 1, 2
bw_builder = RelativisticBreitWignerBuilder(
    energy_dependent_width=False,
    form_factor=False,
)
for name in reaction.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, bw_builder)
model = model_builder.formulate()

The first component of the full amplitude is shown below.
The other terms are in similar way of formualation and thus not showing explicitly here again.

In [ ]:
(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

In [ ]:
sorted_parameter_defaults = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
}
src = aslatex(sorted_parameter_defaults)
Math(src)

In [ ]:
Math(aslatex(model.kinematic_variables))

## Visualization

In [ ]:
unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="jax",
)

In [ ]:
phsp_event = 500_000
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(phsp_event, rng)

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables,
    backend="jax",
)
phsp = helicity_transformer(phsp_momenta)

In [ ]:
%config InlineBackend.figure_formats = ['png']

fig, ax = plt.subplots(dpi=200)
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
    weights=intensity_func(phsp),
)
ax.set_title("Model-weighted Phase space Dalitz Plot")
ax.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
resonances = defaultdict(set)
for transition in reaction.transitions:
    topology = transition.topology
    top_decay_products = topology.get_edge_ids_outgoing_from_node(0)
    (resonance_id, resonance), *_ = transition.intermediate_states.items()
    recoil_id, *_ = top_decay_products - {resonance_id}
    resonances[recoil_id].add(resonance.particle)
resonances = {k: sorted(v, key=lambda p: p.mass) for k, v in resonances.items()}
{k: [p.name for p in v] for k, v in resonances.items()}

In [ ]:
%config InlineBackend.figure_formats = ['svg']
fig, axes = plt.subplots(figsize=(14, 4), ncols=3, sharey=True)
ax1, ax2, ax3 = axes

ax1.set_xlabel(R"$m(\pi^0 p)$ [GeV]")
ax2.set_xlabel(R"$m(\eta p)$ [GeV]")
ax3.set_xlabel(R"$m(\eta \pi^0)$ [GeV]")

color_id = 0
intensities = intensity_func(phsp)
for recoil_id, ax in enumerate(axes):
    decay_products = sorted({0, 1, 2} - {recoil_id})
    key = f"m_{''.join(str(i) for i in decay_products)}"
    ax.hist(
        phsp[key].real,
        alpha=0.5,
        bins=100,
        density=True,
        weights=intensities,
    )
    for resonance in resonances[recoil_id]:
        ax.axvline(
            resonance.mass,
            c=f"C{color_id}",
            ls="dotted",
            label=resonance.name,
        )
        color_id += 1
    ax.legend()

fig.tight_layout()
plt.show(fig)